In [2]:
from decodes.core import *
from decodes.io.jupyter_out import JupyterOut
import pprint

pp = pprint.PrettyPrinter(indent=4)
out = JupyterOut.unit_square( )
item = "item"

http://decod.es/	v0.2.3
io loaded


# Multi-Dimensionality

Data often arrives in arrangements of more than one parameter, suggesting a representation in more than one dimension. This section presents an overview of multi-dimensional collections, which will allow us to: 
* represent data that is naturally structured in more than one dimension
* combine different types of collections in such a way as to leverage the natural advantages of each

A multi-dimensional collection can be defined as one that arranges groupings of data in ***more than one dimension***, like the rows and columns of a spreadsheet, or that ***embeds one grouping inside another***, like the directory tree structure that governs the location of folders and files on our computer.

Working between these two general models for a multi-dimensional collection - ***the spreadsheet and the directory tree*** - this discussion distinguishes these two models, presents what alternatives exist as hybrids between them, and what the implications of these distinctions might be in a design context.





<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.04.P23.jpg" style="width: 600px; display: inline;">

* A **matrix** is an n-dimensional array that organizes data into regular rows and columns
* A **nest** is a collection of arrays that might contain rows of irregular dimension
* A **tree** is a multi-level collection that may contain items, or deeper collections at any level.

## Multiple Paths to Higher Dimensions

We begin by exploring data in a form that is easily visualized: two-dimensions. 

The ***dimension*** of a collection, a quantity termed the collection’s ***rank***, may be determined by identifying ***the number of indices required to select a single object in that collection***.

Imagine that we are working with a spreadsheet, with data arranged in rows and columns. In order to select the data contained within a single cell, two values are required: one specifying a row, and one a column. Unambiguously, The ***rank of a spreadsheet is two***. 

Now imagine that we are working with a typical directory tree, with data (files) arranged in folders. In contrast with the spreadsheet, this amount of information alone is not sufficient to ascertain the rank of this folder structure, which remains ambiguous without further information.

For example, if we were to specify that the directory tree contains three folders at the top-level, each of which contains exactly four files, then we would be able to determine the rank easily. To select a file, we would need to know two pieces of information: the name of the file, and the name of its containing folder. Two required indices reveals ***the rank of this constrained directory tree is two***, the same as our spreadsheet.


<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.04.P21.jpg" style="width: 200px; display: inline;"><img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.04.P22.jpg" style="width: 200px; display: inline;">

While ***the very structure of the spreadsheet determines its rank***, the directory tree requires additional information for the rank to be determined: ***Our two-level directory structure could just as easily been five-level***. 

At times, ***the rank of a directory tree may be impossible to determine***: if files were to be mixed in with folders at various levels, then a variable number of indices would be required, and would depend on the specific location of the file in question. One file at the top-level of the structure may only require a single index, while another deep in a sub-folder may require five. *Some combination of indices would direct us to a single file, while others would direct us to an entire folder*.

The spreadsheet and the directory structure suggest contrasting models of a multi-dimensional collection. 
* The regular structure of the spreadsheet is a **matrix**.
* The stratified levels possible in a directory structure is a **tree**.

A look at the implementation of multi-dimensional collections in code will allow us to understand a third model of collection that lies between the poles of matrix and tree.

### Nests in Python

Many languages require that the size, number of dimensions, and contained type of a multi-dimensional collection be stated upon construction. Since Python neither requires initialization of collections nor the assignment of an allowed type, there is no exact equivalent for the construction of an empty List or "matrix" of a fixed size.

In C#, for example, we could construct an empty two-dimensional array capable of containing Points as such:

    Point[,] pointGridCSharp = new Point[n,m];
    
This would result in a collection that might be diagramed as shown below.

The selection of a particular item in such an array would appear as:

    pointGridCSharp[3,2];

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.04.P15.jpg" style="width: 200px; display: inline;">

We can see from the syntax of construction and the related diagram that the two-dimensional array in C# appears to be a matrix: a single entity that addresses elements by a pair of numbers, and that houses precisely `n` rows and `m` columns. The number of rows and columns is ***fixed and regular***. The ***number of indices is baked into the assignment syntax*** given by square-brackets containing two values. It would be***impossible to select a row of items*** in a single expression.

The configuration of the this collection has been explicitly set before the addition of any items, and through this mechanism, ***an equal number of columns in each row is enforced***.

In Python, we can approximate a grid-like matrix structure as seen in the code below, by iteratively constructing Lists that are each appended to a larger List. The corresponding expression to select a single Point would appear
as: 

    pt_grid_python[3][2]

In [5]:
"""
A Grid of Points
In contrast with other languages such as C#, Python Lists are not typed and 
not sized at construction. To create a List of a given size, we must fill 
it with elements.
"""
pt_grid_python = []
for ni in range(4):
    row = []
    for mi in range(3): row.append(Point(ni,mi))
    pt_grid_python.append(row)
    
# pp.pprint(pt_grid_python)

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.04.P16.jpg" style="width: 200px; display: inline;">

Compare the Python syntax of construction and related diagram to the C# above.

The structure in Python appears to be a construction of ***nested Lists***: a single top-level List that encloses `n` number of contained Lists of `m` items each. The syntax in this case, a separate square-bracket enclosure for each index, displays a natural separation of rows from columns. This suggests that we could select an entire row of items by simply leaving off an index, and we find that `pt_grid_python[3]` is indeed a valid expression that returns a List.

Unlike the C# version above, ***an equal number of columns in each row is not enforced***, and this collection might just as easily have appended a varying number of items to each contained List.

In [4]:
print(pt_grid_python[3])

[pt[3,0,0], pt[3,1,0], pt[3,2,0]]


This comparison shows us another way we may characterize a multi-dimensional collection. 

A collection is said to be ***rectangular*** if every row contains the same number of columns, and is said to be ***jagged*** if it does not. 

The above C# array, like all matrices, is rectangular by definition. The Python collection makes no such distinction. The example above happens to be rectangular, but holds the potential to become jagged the moment a varying number of items were to appear in one of its rows, as shown in the code below.

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.04.P17.jpg" style="width: 200px; display: inline;">

In [6]:
"""
Nest
We define a as a non-rectangular or "jagged" array in which each row of 
objects may contain a different number of items. Sometimes termed a "list 
of lists", a nest maintains a consistent dimension, and, unlike a tree, 
does not store items at multiple levels.
"""
nest = []
nest.append( [item,item,item,item,item,item,item,item] ) # nest[0]
nest.append( [item,item,item] ) # nest[1]
nest.append( [item,item,item,item] ) # nest[2]
nest.append( [item,item,item,item,item,item,item,item] )# nest[3]
nest.append( [item,item] ) # nest[4]

pp.pprint(nest)

[   ['item', 'item', 'item', 'item', 'item', 'item', 'item', 'item'],
    ['item', 'item', 'item'],
    ['item', 'item', 'item', 'item'],
    ['item', 'item', 'item', 'item', 'item', 'item', 'item', 'item'],
    ['item', 'item']]


This structure is widely referred to as a ***“list-of-lists”***. In Python, since any sort of collection may be nested inside another (and is not limited to Lists), we instead apply the term ***nest***. 

A nest exhibits qualities of both matrix and tree, and therefore lies somewhere in between. 

Like a matrix, ***a nest exhibits a clear rank***, and allows the selection of items using exactly two indices. Unlike a matrix, however, ***the valid range of an index is variable***, and depends upon the specific arrangement of preceding indices.

Like a tree, a nest shows flexibility in the number of items at a given level. Unlike a tree, we do not find items at multiple levels.

In [12]:
# for the given Nest, this is okay
print( nest[2][3] )

# for the given Nest, this is not okay
try:
    print( nest[1][3] ) 
except:
    print( "nope" )

item
nope


### Trees in Python

We define a tree as a multi-level collection that may contain both items or deeper collections at any level. As such, similar configurations of indices may alternately return items or collections of items. 

Consider a tree similar in structure to the nest defined above. Like a nest, Python does not require any special syntax in order to construct a tree.

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.04.P18.jpg" style="width: 600px; display: inline;">

In [14]:
"""
Tree
A tree is a multi-level collection that is non-rectangular, and that mixes 
objects and collections at a variety of levels. Unlike a nest, it is not 
possible to determine the rank of a tree.
"""
tree = []
tree.append( [item,item,item,item,item,item,item,item] )
tree.append( item )
tree.append( [item,item,item,item] )
tree.append( [item,item,item,item,item,item,item,item] )
tree.append( [ [item,item] ,item] )

pp.pprint(tree)

[   ['item', 'item', 'item', 'item', 'item', 'item', 'item', 'item'],
    'item',
    ['item', 'item', 'item', 'item'],
    ['item', 'item', 'item', 'item', 'item', 'item', 'item', 'item'],
    [['item', 'item'], 'item']]


We can see from the diagram that ***the relationship between configurations of indices and returned objects is varied***.

Expressions containing a single index are valid, but return a variety of results: `tree[1]` returns an item, while `tree[3]` returns a collection of eight items. 

In only some cases, a configuration of three indices may be valid, such as `tree[4][0][1]` which also returns a single item.

In [15]:
print( tree[1] )
print( tree[3] )

print( tree[4][0][1] )

item
['item', 'item', 'item', 'item', 'item', 'item', 'item', 'item']
item


Given the variety of index configurations possible in a tree, we tend to speak about item access not in terms of dimension or rank, but in terms of the particular path that leads to a desired item in a tree:

* If a path returns a collection then we call it a ***branch***. 
* If it returns an item then we call it a ***leaf***. 
* The base path is considered the tree’s ***root***.

## A Glossary of Multi-Dimensional Collections

**Rank**
> The number of indices that are required to specify an item in an array. For flat collections such as matrices, and simple jagged arrays such as nests, this property may be determined. Rank is indeterminate in the case of trees.

**Rectangular vs Jagged**
> A collection is rectangular if the range of valid indices is fixed across dimensions, and jagged if it is not. In other words, the collection is rectangular if the number of columns per row is consistent and jagged if it is not. A matrix is a rectangular array, and a nest is a jagged array.

**Homogeneous vs Heterogeneous**
> A homogeneous collection contains only one type of object, while a heterogeneous collection contains a variety of types. While some languages require that collections declare a fixed type, Python does not impose such a restriction.

**Fixed vs Dynamically-Sized**
> A fixed-size collection can only contain a predefined number of objects, while a dynamically-sized collection expands to fit however many elements are added. In Python, all collections are dynamically-sized.
